## Fit Hedonic Regression Models 

This notebook fits the hedonic regression models for the paper titled 'Using Hedonic Modelling to Assess Spatial Inequalities of Air Conditioning in Madrid'. 

In [1]:
# Read in libraries 
import pandas as pd
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt
import pysal
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
from pysal.model import spreg
from shapely import Point
import seaborn as sns
from pysal.model import spreg
import statsmodels
import statsmodels.formula.api as smf

/opt/conda/envs/gds/lib/python3.9/site-packages/spglm/utils.py:367: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if resetlist is not ():
/opt/conda/envs/gds/lib/python3.9/site-packages/spvcm/utils.py:149: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if np.isinf(ldet) or sgn is 0:
/opt/conda/envs/gds/lib/python3.9/site-packages/spvcm/abstracts.py:268: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if chains is () and kwargs != dict():
/opt/conda/envs/gds/lib/python3.9/site-packages/spvcm/abstracts.py:270: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if chains is not ():
/opt/conda/envs/gds/lib/python3.9/site-packages/spvcm/plotting.py:37: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if thin is None or thin is 0:


In [2]:
# Function to calculate BIC

def calculate_bic(model, data):
    """
    Calculate the Bayesian Information Criterion (BIC) for a fitted model.
    
    Parameters:
        model: A fitted model object (e.g., PySAL OLS or statsmodels OLS).
        data: The dataset used to fit the model.
    
    Returns:
        float: The calculated BIC value.
    """
    # Number of observations
    n = len(data)

    # Number of parameters (including the intercept)
    k = len(model.betas)

    # Residuals (difference between observed and predicted values)
    residuals = model.u  # For PySAL's OLS model, residuals are stored in 'u'

    # Residual sum of squares (RSS)
    rss = np.sum(residuals ** 2)

    # Calculate BIC
    bic = n * np.log(rss / n) + k * np.log(n)

    return bic


In [3]:
data = pd.read_csv('df_new2finalmodeldata.csv')

## Fit the regression models 

### Model 1: Including only HAS AIR CONDITIONING predictor 

In [287]:
# Fit OLS model
m1 = spreg.OLS(
    # Dependent variable
    data[["Log_UNITPRICE"]].values,
    # Independent variables
    data[["HASAIRCONDITIONING"]].values,
    # Dependent variable name
    name_y="Log_UNITPRICE",
    # Independent variable name
    name_x=["HASAIRCONDITIONING"],
)

# Print summary of the model
print(m1.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :Log_UNITPRICE                Number of Observations:      578090
Mean dependent var  :      7.8868                Number of Variables   :           2
S.D. dependent var  :      0.5113                Degrees of Freedom    :      578088
R-squared           :      0.0493
Adjusted R-squared  :      0.0493
Sum squared residual:      143699                F-statistic           :  29963.5985
Sigma-square        :       0.249                Prob(F-statistic)     :           0
S.E. of regression  :       0.499                Log likelihood        : -417921.648
Sigma-square ML     :       0.249                Akaike info criterion :  835847.297
S.E of regression ML:      0.4986                Schwarz criterion     :  835869.832

-----------------------------------------------------------

In [289]:
bic_value = calculate_bic(m1, data)
print(f"BIC: {bic_value}")

BIC: -804678.5214446082


### Model 2: Add in all structural predictor variables

In [ ]:
# List structural variables 
variable_names_s = ['ROOMNUMBER', 'HASTERRACE', 'HASLIFT','HASAIRCONDITIONING','HASPARKINGSPACE',
                    'PARKINGSPACEPRICE', 'HASNORTHORIENTATION', 'HASSOUTHORIENTATION', 'HASEASTORIENTATION', 'HASWESTORIENTATION',
                     'HASSWIMMINGPOOL', 'HASDOORMAN', 'HASGARDEN','ISDUPLEX','ISSTUDIO','ISINTOPFLOOR',
                    'FLOORCLEAN', 'CADMAXBUILDINGFLOOR','CADDWELLINGCOUNT', 'CoolingDemandCategory'

]

In [37]:
## Linear regression model-add in structural variables 

# Fit OLS model
m2 = spreg.OLS(
    # Dependent variable
    data[["Log_UNITPRICE"]].values,
    # Independent variables
    data[variable_names_s].values,
    # Dependent variable name
    name_y="Log_UNITPRICE",
    # Independent variable name
    name_x=variable_names_s,
)

# Print summary of the model
print(m2.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :Log_UNITPRICE                Number of Observations:      578080
Mean dependent var  :      7.8868                Number of Variables   :          21
S.D. dependent var  :      0.5113                Degrees of Freedom    :      578059
R-squared           :      0.2754
Adjusted R-squared  :      0.2754
Sum squared residual:      109524                F-statistic           :  10984.0168
Sigma-square        :       0.189                Prob(F-statistic)     :           0
S.E. of regression  :       0.435                Log likelihood        : -339423.167
Sigma-square ML     :       0.189                Akaike info criterion :  678888.334
S.E of regression ML:      0.4353                Schwarz criterion     :  679124.950

-----------------------------------------------------------

In [38]:
bic_value = calculate_bic(m2, data)
print(f"BIC: {bic_value}")

BIC: -961395.0241264665


### Model 3: Add in all the predictor variables 

In [39]:
# List structural variables 
variable_names = [ 'ROOMNUMBER', 'HASTERRACE', 'HASLIFT','HASAIRCONDITIONING','HASPARKINGSPACE',
                    'PARKINGSPACEPRICE', 'HASNORTHORIENTATION', 'HASSOUTHORIENTATION', 'HASEASTORIENTATION', 'HASWESTORIENTATION',
                     'HASSWIMMINGPOOL', 'HASDOORMAN', 'HASGARDEN','ISDUPLEX','ISSTUDIO','ISINTOPFLOOR',
                    'FLOORCLEAN', 'CADMAXBUILDINGFLOOR','CADDWELLINGCOUNT', 'CoolingDemandCategory','log_income',
                    'building_density', 'bench_density','green_area_percent'

]

In [40]:
### Add in neighborurhood variables 
## Linear regression model-add in structural variables 

# Fit OLS model
m3 = spreg.OLS(
    # Dependent variable
    data[["Log_UNITPRICE"]].values,
    # Independent variables
    data[variable_names].values,
    # Dependent variable name
    name_y="Log_UNITPRICE",
    # Independent variable name
    name_x=variable_names,
)

# Print summary of the model
print(m3.summary).T

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :Log_UNITPRICE                Number of Observations:      578080
Mean dependent var  :      7.8868                Number of Variables   :          25
S.D. dependent var  :      0.5113                Degrees of Freedom    :      578055
R-squared           :      0.5753
Adjusted R-squared  :      0.5752
Sum squared residual:     64198.5                F-statistic           :  32620.8694
Sigma-square        :       0.111                Prob(F-statistic)     :           0
S.E. of regression  :       0.333                Log likelihood        : -185027.365
Sigma-square ML     :       0.111                Akaike info criterion :  370104.729
S.E of regression ML:      0.3332                Schwarz criterion     :  370386.416

-----------------------------------------------------------

AttributeError: 'NoneType' object has no attribute 'T'

In [41]:
bic_value = calculate_bic(m3, data)
print(f"BIC: {bic_value}")

BIC: -1270133.5584945134


### Model 4, 5, 6 add in interaction terms

In [42]:
### Fit the interaction effect models
# Create an interaction term: HASAIRCONDITIONING * HASSOUTHORIENTATION
data["INTERACTION_TERM"] = data["HASAIRCONDITIONING"] * data["log_income"]

# Add the interaction term to the variable_names list
variable_names_with_interaction = variable_names + ["INTERACTION_TERM"]

# Fit OLS model with interaction term
m4 = spreg.OLS(
    # Dependent variable
    data[["Log_UNITPRICE"]].values,
    # Independent variables
    data[variable_names_with_interaction].values,
    # Dependent variable name
    name_y="Log_UNITPRICE",
    # Independent variable name
    name_x=variable_names_with_interaction,
)

# Print summary of the model
print(m4.summary)


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :Log_UNITPRICE                Number of Observations:      578080
Mean dependent var  :      7.8868                Number of Variables   :          26
S.D. dependent var  :      0.5113                Degrees of Freedom    :      578054
R-squared           :      0.5759
Adjusted R-squared  :      0.5759
Sum squared residual:     64106.1                F-statistic           :  31394.3997
Sigma-square        :       0.111                Prob(F-statistic)     :           0
S.E. of regression  :       0.333                Log likelihood        : -184611.296
Sigma-square ML     :       0.111                Akaike info criterion :  369274.592
S.E of regression ML:      0.3330                Schwarz criterion     :  369567.546

-----------------------------------------------------------

In [17]:
bic_value = calculate_bic(m4, data)
print(f"BIC: {bic_value}")

BIC: -1273154.0982677215


In [13]:
### Fit the interaction effect models
# Create an interaction term: HASAIRCONDITIONING * HASSOUTHORIENTATION
data["INTERACTION_TERM"] = data["HASAIRCONDITIONING"] * data["building_density"]

# Add the interaction term to the variable_names list
variable_names_with_interaction = variable_names + ["INTERACTION_TERM"]

# Fit OLS model with interaction term
m5 = spreg.OLS(
    # Dependent variable
    data[["Log_UNITPRICE"]].values,
    # Independent variables
    data[variable_names_with_interaction].values,
    # Dependent variable name
    name_y="Log_UNITPRICE",
    # Independent variable name
    name_x=variable_names_with_interaction,
)

# Print summary of the model
print(m5.summary)


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :Log_UNITPRICE                Number of Observations:      578080
Mean dependent var  :      7.8868                Number of Variables   :          28
S.D. dependent var  :      0.5113                Degrees of Freedom    :      578052
R-squared           :      0.5775
Adjusted R-squared  :      0.5775
Sum squared residual:     63858.5                F-statistic           :  29264.5089
Sigma-square        :       0.110                Prob(F-statistic)     :           0
S.E. of regression  :       0.332                Log likelihood        : -183492.757
Sigma-square ML     :       0.110                Akaike info criterion :  367041.513
S.E of regression ML:      0.3324                Schwarz criterion     :  367357.002

-----------------------------------------------------------

In [18]:
bic_value = calculate_bic(m5, data)
print(f"BIC: {bic_value}")

BIC: -1272397.9576739648


In [19]:
### Fit the interaction effect models
# Create an interaction term: HASAIRCONDITIONING * HASSOUTHORIENTATION
data["INTERACTION_TERM"] = data["HASAIRCONDITIONING"] * data["CoolingDemandCategory"]

# Add the interaction term to the variable_names list
variable_names_with_interaction = variable_names + ["INTERACTION_TERM"]

# Fit OLS model with interaction term
m6 = spreg.OLS(
    # Dependent variable
    data[["Log_UNITPRICE"]].values,
    # Independent variables
    data[variable_names_with_interaction].values,
    # Dependent variable name
    name_y="Log_UNITPRICE",
    # Independent variable name
    name_x=variable_names_with_interaction,
)

# Print summary of the model
print(m6.summary)


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :Log_UNITPRICE                Number of Observations:      578080
Mean dependent var  :      7.8868                Number of Variables   :          28
S.D. dependent var  :      0.5113                Degrees of Freedom    :      578052
R-squared           :      0.5769
Adjusted R-squared  :      0.5769
Sum squared residual:     63943.1                F-statistic           :  29197.4930
Sigma-square        :       0.111                Prob(F-statistic)     :           0
S.E. of regression  :       0.333                Log likelihood        : -183875.264
Sigma-square ML     :       0.111                Akaike info criterion :  367806.528
S.E of regression ML:      0.3326                Schwarz criterion     :  368122.017

-----------------------------------------------------------

In [20]:
bic_value = calculate_bic(m6, data)
print(f"BIC: {bic_value}")

BIC: -1272397.9576739648


## Prepare the data for the multilevel model

In [26]:
# Example: Select only specific columns
barrios = gpd.read_file('barrios.shp')
data = gpd.GeoDataFrame(data, geometry=data['geometry'], crs=25830)
joined = gpd.sjoin(data, barrios, how="inner", predicate="within")

In [27]:
joined.columns = [col..replace("_", "").replace(".", "") for col in joined.columns]

In [43]:
# Save df_new2 to a CSV file
df_new2.to_csv('final_data_with.csv', index=False)